In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)
from sklearn.preprocessing import StandardScaler
import numpy as np
from tkinter import *                    
from tkinter import messagebox as mb      
from tkinter import filedialog as fd
from sklearn.cluster import AgglomerativeClustering

In [12]:
global count, loaded_filename, labels, main_df
labels = {}
count = 0

def clusterization_get_labels(df):
        global labels
        df1 = df.copy(deep=True)
        df1 = df1.drop(columns='Meter Configuration ID')
        df1 = df1.iloc[: , 1:]
        df1['SIC_Code'] = df1['SIC_Code'].astype(str)
        columns = list(df1.columns[6:-1])
        scaler = StandardScaler()
        new_data =  pd.DataFrame(scaler.fit_transform(df1[columns]), 
                       columns=columns, index=df1.index)
        new_df = df1.copy(deep=True)
        new_df[columns] = new_data
        display(new_df)
        
        hierarchal = AgglomerativeClustering(n_clusters = 3).fit(pd.get_dummies(new_df))
        df['cluster'] = hierarchal.labels_
        labels = df[['SA ID', 'cluster']]
        labels['cluster'] = labels['cluster'] + 1
        labels = dict(labels.values)
        return df



def clusterization():
    
    global df_upd, main_df, labels
    df_upd = clusterization_get_labels(df_preprocessed)
    
    L1 = Label(window, height=1, width=150, text='Данные были кластеризированы.')
    L2 = Label(window, height=2, width=150, text='Результат выгружен в: ' + loaded_filename[:-5] + "_clusterization_result.xlsx")
    main_df['cluster'] = '-'
    for t in labels.keys():
        main_df.loc[main_df['SA ID'] == t, ['cluster']] = labels[t]
    data_transfer = main_df.copy(deep=True)
    data_transfer.insert(1, column='тип потребителя(номер кластера)', value=data_transfer['cluster'])
    data_transfer.drop(columns=['cluster']).to_excel(loaded_filename[:-5] + "_clusterization_result.xlsx")
    
    
    L1.pack(pady=8)
    L2.pack(pady=8)
    
    column_names = list(df_upd.drop(columns='Meter Configuration ID').iloc[: , 1:-1])
    f_top.pack(pady=8)
    for i in column_names:
            x_b.insert(END, i)
            y_b.insert(END, i)
    x_label.pack(side='left',pady=8)
    x_b.pack(side='left',pady=8)
    y_label.pack(side='right',pady=8)
    y_b.pack(side='right',pady=8)
    
    
    button_plot.pack(pady=8)
    button_statistics.pack(pady=8)

def zagruzka_bazi():
    
    global loaded_filename
    global count, L
    loaded_filename = fd.askopenfilename(initialdir = "/",
                                          title = "Select a File",
                                          filetypes = (("xlsx файлы",
                                                        ".xlsx"),
                                                      ))
    
    if loaded_filename:
        count+=1
        L.config(text='Загружена база: ' + loaded_filename)
        L.pack(pady=8)
        
        button_predobrabotka.pack(pady=8)
        
        if count > 0:
            list = window.winfo_children()
            for i in range(3, len(list)):
                list[i].destroy()
                
                
                global button_clusterization, button_plot, button_statistics 
                global f_top, x_label, x_b, y_label, y_b, x_scroll, y_scroll
                button_clusterization = Button(window,
                        text = "Кластеризировать",
                        command = clusterization)
                button_plot = Button(window,
                        text = "Построить график по заданным параметрам",
                        command = plot)

                button_statistics = Button(window, text = 'Получить статистику по результатам кластеризации', command = statistics)
                f_top = Frame(window)
                x_label = Label(f_top, text='Признак по x')
                x_b = Listbox(f_top, width=25, exportselection=False)
                y_label = Label(f_top, text='Признак по y')
                y_b = Listbox(f_top, width=25, exportselection=False)
                x_scroll = Scrollbar(command=x_b.yview)
                y_scroll = Scrollbar(command=y_b.yview)
                

                
    return loaded_filename



def preprocessing_data(file_name):
        global main_df
        df = pd.read_excel(loaded_filename)
        main_df = df.copy(deep=True)
        n_row = df.shape[0]
    
        columns_name = df.columns.tolist()
        sub = 'Unnamed:'
        df = df.drop([s for s in columns_name if sub in s], axis = 1).copy(deep = True) 
    
        first_note = 0
        months_name = [
            'январь',
            'февраль',
             'март',
             'апрель',
             'май',
             'июнь',
             'июль',
             'август',
             'сентябрь',
             'октябрь',
             'ноябрь',
             'декабрь']
        for s in columns_name:
            if any(ext in s for ext in months_name):
                break
            first_note += 1
    
        n_na_col = n_row / 3 # кол-во NA значений в столбце, больше которого мы хотим удалять столбец
    
        df = df.dropna(axis = 1, thresh = n_row - n_na_col).copy(deep = True)
        df[df.isna().any(axis=1)].to_excel(loaded_filename[:-5] + "_unused_values.xlsx")
        df = df.dropna(axis=0).copy(deep = True)
    
        return df

    
def make_df_dif_month(df):
        columns_name = df.columns.tolist()
        first_note = 0
        months_name = [
             'январь',
             'февраль',
             'март',
             'апрель',
             'май',
             'июнь',
             'июль',
             'август',
             'сентябрь',
             'октябрь',
             'ноябрь',
             'декабрь']
        for st in columns_name:
            if any(ext in st for ext in months_name):
                break
            first_note += 1    
        s = 0
        amount_positive = 0
        df_dif_month = df.copy(deep = True)
        df_dif_month = df_dif_month.reset_index().drop(columns = "index").copy(deep = True)
        n_row = df_dif_month.shape[0]
        dates = df_dif_month.columns.tolist()[first_note:] # список дат
        date_dif_list = list()
        for i in range(len(dates) - 1):
            date_dif = dates[i + 1].split()[0] + " " + dates[i].split()[1] + " - " + dates[i + 1].split()[1] # конструируем строку типа "год месяц - месяц"
            date_dif_list.append(date_dif)
            df_dif_month[date_dif] = (df_dif_month[dates[i + 1]] - df_dif_month[dates[i]]).copy(deep = True) # добавляем столбец в датафрейм с разностью показаний счётчиков
        df_dif_month = df_dif_month.drop(columns = dates).copy(deep = True) # удаляем заменяемые столбцы
    
        for i in range(n_row):
            for j in range(len(date_dif_list)):
                if(df_dif_month[date_dif_list[j]][i] >= 0):
                    s += df_dif_month[date_dif_list[j]][i]
                    amount_positive += 1
            avarage = s / (amount_positive)
        
            for k in range(len(date_dif_list)):
                if ((df_dif_month[date_dif_list[k]][i] >= 21 * avarage) 
                and (df_dif_month[date_dif_list[k]][i] >= 10000)):
                    avarage = (avarage * (amount_positive / (amount_positive - 1)) 
                           - df_dif_month[date_dif_list[k]][i] / (amount_positive - 1))
                
            for l in range(len(date_dif_list)):
                if (df_dif_month[date_dif_list[l]][i] < 0 or 
                ((df_dif_month[date_dif_list[l]][i] >= 21 * avarage) 
                and (df_dif_month[date_dif_list[l]][i] >= 10000))): 
                     df_dif_month[date_dif_list[l]][i] = avarage
        
            s = 0
            amount_positive = 0 
        df_dif_month.to_excel(loaded_filename[:-5]+"_preprocessed.xlsx")    
        return df_dif_month

def predobrabotka():
    
    my_df = preprocessing_data(loaded_filename)
    
    global df_preprocessed
    df_preprocessed = make_df_dif_month(my_df)
    
    L1 = Label(window, height=1, width=150, text='Данные были предобработаны.')
    L2 = Label(window, height=2, width=150, text='Результат выгружен в: ' + loaded_filename[:-5] + "_preprocessed.xlsx")
    L3 = Label(window, height=2, width=150, text='Данные, удаленные при предобработке, выгружены в: '
               + loaded_filename[:-5] + "_unused_values.xlsx")
    L1.pack(pady=8)
    L2.pack(pady=8)
    L3.pack(pady=8)
    
    button_clusterization.pack(pady=8)
    
    
def clusterization_graph(df, x_value, y_value):
    
        fig = Figure(figsize = (14,14))
        plot = fig.add_subplot(111)
        colors = ['red', 'blue', 'green']
        df['c'] = df.cluster.map({0:colors[0], 1:colors[1], 2:colors[2] })
        for t in colors:
            plot.scatter(df[df.c==t][x_value], df[df.c==t][y_value], c=t,label=t, alpha=0.6, s=10)
        plot.set_title('Распределение потребителей в результате кластеризации'+
                  ' в зависимости от признаков: ' + x_value + '; ' + y_value)
        plot.legend(['потребитель электроэнергии 1 типа','потребитель электроэнергии 2 типа',
                    'потребитель электроэнергии 3 типа'])
        plot.set_xlabel(x_value)
        plot.set_ylabel(y_value)
        
        
        okno = Tk()
  

        okno.title('График')
  

        okno.geometry("1000x1000")
    
        canvas = FigureCanvasTkAgg(fig,
                               master = okno)  
        canvas.draw()
  
        canvas.get_tk_widget().pack(expand=True)
  
        toolbar = NavigationToolbar2Tk(canvas,
                                   okno)
        toolbar.update()
  
        canvas.get_tk_widget().pack()
    
    
def plot():
    for i in x_b.curselection():
        x_value = x_b.get(i)
    for i in y_b.curselection():
        y_value = y_b.get(i)
    if x_value and y_value:
        clusterization_graph(df_upd, x_value, y_value)   
    button_plot.pack(pady=8)
                
def statistics():
    
    global df_upd
    
    df = df_upd.copy(deep=True)
    df = df.drop(columns='Meter Configuration ID')
    df = df.iloc[: , 1:]
    s = 'cluster'
    
    avgs = [np.mean(df[df[s]==t].iloc[:,6:-2].mean(axis=1).to_list()) for t in range(0,3)]
    amounts = [len(df[df[s]==t]) for t in range(0,3)]
    label_avg = pd.DataFrame({'номер кластера(тип потребителя)' : [1,2,3], 'среднее по показаниям': avgs,
                         'количество потребителей этого типа(количество элементов в кластере)': amounts})
    label_avg.to_excel(loaded_filename[:-5] + "_statistics.xlsx")
    
    L2 = Label(window, height=2, width=150, text='Статистика выгружена в: ' + loaded_filename[:-5] + "_statistics.xlsx")
    L2.pack(pady=8)

#Основная программа
s = Tk()
  

s.title('Программа')
  

s.geometry("1000x800")
  

s.configure(bg = "#D8E9E6")


window = Frame(s, bg = "#D8E9E6")
window.pack(side="top", expand=True, fill="both")

button_zagruzka = Button(window,
                        text = "Загрузить базу данных",
                        command = zagruzka_bazi)

button_predobrabotka = Button(window, text = "Предобработать данные", command = predobrabotka)

L = Label(window, height=2, width=150)

button_clusterization = Button(window,
                        text = "Кластеризировать",
                        command = clusterization)

button_plot = Button(window,
                        text = "Построить график по заданным параметрам",
                        command = plot)

button_statistics = Button(window, text = 'Получить статистику по результатам кластеризации', command = statistics)
   
button_zagruzka.pack(pady=8)

f_top = Frame(window)
x_label = Label(f_top, text='Признак по x')
x_b = Listbox(f_top, width=25, exportselection=False)
y_label = Label(f_top, text='Признак по y')
y_b = Listbox(f_top, width=25, exportselection=False)
x_scroll = Scrollbar(command=x_b.yview)
y_scroll = Scrollbar(command=y_b.yview)

s.mainloop()